In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from sklearn.mixture import GaussianMixture
from sklearn import metrics
import random
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import figure
from sklearn.cluster import KMeans
from scipy.special import expit
import torch
from torch import nn, optim
import torch.nn.functional as F
from datasets.datasets import get_MNIST_subset_dataloader
from typing import Optional
from scipy.optimize import linear_sum_assignment as linear_assignment

from sklearn.manifold import TSNE

cuda = True if torch.cuda.is_available() else False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device

device(type='cuda', index=0)

## Mnist

In [3]:
n_clusters = 10 # 10
batch_size = 256
data_per_pattern = 2000
latent_dim = 50
create_subset = True
dataloader = get_MNIST_subset_dataloader(batch_size=batch_size, create_subset=create_subset, data_per_pattern=data_per_pattern)
N = len(dataloader) * batch_size

### MLP ARCHITECTURE

In [4]:
h1 = 500 # 500
h2 = 500 # 500
h3 = 2000 # 2000
latent_dim = 10

class Autoencoder(nn.Module):
    def __init__(self, data_shape):
        super(Autoencoder, self).__init__()
        # Encoder Model
        self.encoder_model = nn.Sequential(
            nn.Linear(data_shape, h1, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h1),
            
            nn.Linear(h1, h2, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h2),
            
            nn.Linear(h2, h3, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h3),
            
            nn.Linear(h3, latent_dim, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(latent_dim),
        )
        
        # Clustering MLP
        self.cluster_MLP = nn.Sequential(
            nn.Linear(latent_dim, n_clusters, bias=True),
            #nn.Sigmoid(),
            nn.BatchNorm1d(n_clusters)
        )
        
        # Decoder Model
        self.decoder_model = nn.Sequential(
            nn.Linear(latent_dim, h3, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h3),
            
            nn.Linear(h3, h2, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h2),
            
            nn.Linear(h2, h1, bias=True),
            #nn.ReLU(inplace=True),
            nn.Sigmoid(),
            nn.BatchNorm1d(h1),
            
            nn.Linear(h1, data_shape, bias=True),
            #nn.ReLU(inplace=True)
            nn.Sigmoid()
        )
    
        self.cluster_MLP_params = self.cluster_MLP[0].parameters()
        self.cluster_MLP_batchNorm = self.cluster_MLP[1].parameters()
    
    def forward(self, x):
        x = self.encoder_model(x)
        x = self.decoder_model(x)
        return x

    def forward_softMax(self, x):
        x = self.encoder_model(x)
        x = self.cluster_MLP(x)
        x = F.softmax(x, dim=1)
        return x
    
    def encoder(self, x):
        x = self.encoder_model(x)
        return x
    
    def decoder(self, x):
        x = self.decoder_model(x)
        return x

data_shape = 784
autoencoder = Autoencoder(data_shape)
autoencoder = autoencoder.to(device)

In [5]:
autoencoder.cluster_MLP_batchNorm

<generator object Module.parameters at 0x7f6dfdcdbbf8>

In [6]:
autoencoder.cluster_MLP[0].parameters()

<generator object Module.parameters at 0x7f6df2d14150>

In [7]:
def soft_silhouette(X, soft_clustering, requires_distance_grad=False):
    # No grads at distances
    if requires_distance_grad: X = X.detach()
    distances = torch.cdist(X, X, p=2)
    alphas = torch.matmul(distances, soft_clustering).to(device)
    n_data, n_clusters = alphas.shape
    betas = torch.empty(alphas.shape).to(device)
    
    for i in range(n_data):
        for j in range(n_clusters):
            betas[i][j] = min([alphas[i][x] for x in range(alphas.shape[1]) if x!=j])
    
    # Calculate soft silhouette
    sc = torch.div(torch.sub(betas, alphas), torch.max(alphas, betas))
    s = torch.mean(torch.sum(torch.mul(soft_clustering, sc), dim=1))

    return s

def predictions(clusters):
    clusters = torch.argmax(clusters, dim=1)
    clusters = clusters.cpu().detach().numpy()
    return clusters

def transform_clusters_to_labels(clusters, labels):
    # Get the data clusters based on max neuron
    #clusters = torch.argmax(clusters, dim=1)
    #clusters = clusters.cpu().detach().numpy()

    # Find the cluster ids (labels)
    c_ids = np.unique(clusters)
    #labels = labels.cpu().data.numpy()

    # Dictionary to transform cluster label to real label
    dict_clusters_to_labels = dict()
    
    # For every cluster find the most frequent data label
    for c_id in c_ids:
        indexes_of_cluster_i = np.where(c_id == clusters)
        elements, frequency = np.unique(labels[indexes_of_cluster_i], return_counts=True)
        true_label_index = np.argmax(frequency)
        true_label = elements[true_label_index]
        dict_clusters_to_labels[c_id] = true_label

    # Change the cluster labels to real labels
    for i, element in enumerate(clusters):
        clusters[i] = dict_clusters_to_labels[element]

    return clusters

def cluster_accuracy(y_true, y_predicted, cluster_number: Optional[int] = None):
    """
    Calculate clustering accuracy after using the linear_sum_assignment function in SciPy to
    determine reassignments.

    :param y_true: list of true cluster numbers, an integer array 0-indexed
    :param y_predicted: list of predicted cluster numbers, an integer array 0-indexed
    :param cluster_number: number of clusters, if None then calculated from input
    :return: reassignment dictionary, clustering accuracy
    """
    if cluster_number is None:
        # assume labels are 0-indexed
        cluster_number = (max(y_predicted.max(), y_true.max()) + 1)
    count_matrix = np.zeros((cluster_number, cluster_number), dtype=np.int64)
    for i in range(y_predicted.size):
        count_matrix[y_predicted[i], y_true[i]] += 1

    row_ind, col_ind = linear_assignment(count_matrix.max() - count_matrix)
    reassignment = dict(zip(row_ind, col_ind))
    accuracy = count_matrix[row_ind, col_ind].sum() / y_predicted.size
    return reassignment, accuracy

In [8]:
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)
mse = nn.MSELoss()

latent_data = np.zeros((N, latent_dim))

for epoch in range(50):
    total_rec_loss = 0
    for batch_index, (real_data, labels) in enumerate(dataloader):
        real_data = real_data.to(device)
        real_data = torch.reshape(real_data, (real_data.shape[0], 784))
        reconstruction = autoencoder.forward(real_data)
        loss = mse(real_data, reconstruction)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_rec_loss += loss.item()
        
        # Selecting the nearest samples and save their priors
        code = autoencoder.encoder(real_data).to(device)
        lower = batch_index * batch_size
        upper = (batch_index + 1) * batch_size
        latent_data[lower:upper] = code.cpu().detach().numpy()
    
    print("Epoch: {} Rec: {:.4f} ".format(epoch, total_rec_loss))

Epoch: 0 Rec: 117.7356 
Epoch: 1 Rec: 76.8611 
Epoch: 2 Rec: 72.6392 
Epoch: 3 Rec: 71.3657 
Epoch: 4 Rec: 70.4825 
Epoch: 5 Rec: 69.7092 
Epoch: 6 Rec: 69.3948 
Epoch: 7 Rec: 69.2028 
Epoch: 8 Rec: 69.0455 
Epoch: 9 Rec: 68.9155 
Epoch: 10 Rec: 68.8077 
Epoch: 11 Rec: 68.7217 
Epoch: 12 Rec: 68.6608 
Epoch: 13 Rec: 68.5685 
Epoch: 14 Rec: 68.3989 
Epoch: 15 Rec: 68.2935 
Epoch: 16 Rec: 68.2135 
Epoch: 17 Rec: 68.1410 
Epoch: 18 Rec: 68.0939 
Epoch: 19 Rec: 68.0548 
Epoch: 20 Rec: 67.9611 
Epoch: 21 Rec: 67.8714 
Epoch: 22 Rec: 67.7983 
Epoch: 23 Rec: 67.7405 
Epoch: 24 Rec: 67.6951 
Epoch: 25 Rec: 67.6616 
Epoch: 26 Rec: 67.6474 
Epoch: 27 Rec: 67.6599 
Epoch: 28 Rec: 67.6840 
Epoch: 29 Rec: 67.6600 
Epoch: 30 Rec: 67.6263 
Epoch: 31 Rec: 67.6047 
Epoch: 32 Rec: 67.5858 
Epoch: 33 Rec: 67.5495 
Epoch: 34 Rec: 67.5020 
Epoch: 35 Rec: 67.4568 
Epoch: 36 Rec: 67.4175 
Epoch: 37 Rec: 67.3851 
Epoch: 38 Rec: 67.3594 
Epoch: 39 Rec: 67.3389 
Epoch: 40 Rec: 67.3224 
Epoch: 41 Rec: 67.3090 
E

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=n_clusters)
kmeans = kmeans.fit(latent_data)
kmeans_centers_init = kmeans.cluster_centers_

In [10]:
kmeans_centers_init.shape, autoencoder.cluster_MLP[0].weight.shape

((10, 10), torch.Size([10, 10]))

In [11]:
#kmeans_centers_init.shape
if cuda:
    dtype = torch.float32
else:
    dtype = torch.cuda.FloatTensor

autoencoder.cluster_MLP[0].weight = nn.Parameter(torch.tensor(kmeans_centers_init, dtype=dtype, device=device))

In [12]:
lamda = 1
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)
mse = nn.MSELoss()

latent_data = np.zeros((N, latent_dim))
real_labels = np.zeros((N), dtype=np.int32)
predicted_labels = np.zeros((N), dtype=np.int32)

for epoch in range(20):
    sum_rec_loss = 0
    sum_soft_sihouette = 0
    for batch_index, (real_data, labels) in enumerate(dataloader):
        real_data = real_data.to(device)
        real_data = torch.reshape(real_data, (real_data.shape[0], 784))
        reconstruction = autoencoder.forward(real_data)
        soft_clustering = autoencoder.forward_softMax(real_data).to(device)
        code = autoencoder.encoder(real_data).to(device)
        s = soft_silhouette(code, soft_clustering, requires_distance_grad=True)
    
        rec = mse(reconstruction, real_data)
        loss = 1 - s + lamda * rec
        
        sum_rec_loss += rec.item()
        sum_soft_sihouette += s.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Selecting the nearest samples and save their priors
        lower = batch_index * batch_size
        upper = (batch_index + 1) * batch_size
        latent_data[lower:upper] = code.cpu().detach().numpy()
        real_labels[lower:upper] = labels.cpu().data.numpy()
        predicted_labels[lower:upper] = predictions(soft_clustering) 
    
    cl_accuracy = cluster_accuracy(real_labels, predicted_labels)[1]
    gr_accuracy = accuracy_score(transform_clusters_to_labels(predicted_labels, real_labels), real_labels)
    nmi = normalized_mutual_info_score(real_labels, predicted_labels)
    total_loss = sum_rec_loss + sum_soft_sihouette
    print("Epoch: {} L: {:.4f} Rec: {:.4f} Soft SIL: {:.2f} CL_ACC: {:.2f} GR_ACC: {:.2f} NMI: {:.2f} ".format(epoch, total_loss, sum_rec_loss, sum_soft_sihouette, cl_accuracy, gr_accuracy, nmi))

Epoch: 0 L: 70.5746 Rec: 68.6331 Soft SIL: 1.94 CL_ACC: 0.53 GR_ACC: 0.57 NMI: 0.47 
Epoch: 1 L: 80.3157 Rec: 69.3040 Soft SIL: 11.01 CL_ACC: 0.54 GR_ACC: 0.57 NMI: 0.49 
Epoch: 2 L: 88.8827 Rec: 69.5332 Soft SIL: 19.35 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 3 L: 96.1220 Rec: 69.6209 Soft SIL: 26.50 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 4 L: 102.6905 Rec: 69.6225 Soft SIL: 33.07 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 5 L: 108.4342 Rec: 69.6221 Soft SIL: 38.81 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 6 L: 113.8037 Rec: 69.5913 Soft SIL: 44.21 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 7 L: 117.8637 Rec: 69.5435 Soft SIL: 48.32 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 8 L: 120.8337 Rec: 69.4712 Soft SIL: 51.36 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.52 
Epoch: 9 L: 123.0357 Rec: 69.3955 Soft SIL: 53.64 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.51 
Epoch: 10 L: 123.3449 Rec: 69.3278 Soft SIL: 54.02 CL_ACC: 0.56 GR_ACC: 0.58 NMI: 0.51 
Epoch: 11 L: 124.1804 Rec: 69.2647 Soft SIL: 54

KeyboardInterrupt: 

In [ ]:
# Cluster with TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=30, n_iter=300)
tsne_enc = tsne.fit_transform(latent_data)
#tsne_enc = tsne.fit_transform(real_data)

# Color and marker for each true class
colors = cm.rainbow(np.linspace(0, 1, n_clusters))
markers = matplotlib.markers.MarkerStyle.filled_markers

# Save TSNE figure to file
fig, ax = plt.subplots(figsize=(16, 10))
for iclass in range(0, n_clusters):
    # Get indices for each class
    idxs = real_labels==iclass
    # Scatter those points in tsne dims
    ax.scatter(tsne_enc[idxs, 0], tsne_enc[idxs, 1],
               marker=markers[iclass], c=colors[iclass].reshape(1,-1),
               edgecolor=None, label=r'$%i$'%iclass)
    
figname="Wine Soft Silhouette"
ax.set_title(figname, fontsize=24)
ax.set_xlabel("$x_{1}$", fontsize=20)
ax.set_ylabel("$x_{2}$", fontsize=20)
plt.legend(title="Class", loc='best', numpoints=1, fontsize=18)
plt.tight_layout()
plt.show()
#fig.savefig(figname, facecolor='w')

In [ ]:
autoencoder.cluster_MLP[0].out_features

In [ ]:
autoencoder

In [ ]:
autoencoder.cluster_MLP[0]

In [ ]:
autoencoder.cluster_MLP[0].weight.shape